In [1]:
import networkx as nx
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import pandas as pd
import os

In [2]:
NUMBER_NODES = 9

def getGraph(upperTriangleAdjMatrix):
    dense_adj = np.zeros((NUMBER_NODES, NUMBER_NODES))
    dense_adj = np.zeros((NUMBER_NODES, NUMBER_NODES))
    k = 0
    for i in range(NUMBER_NODES):
        for j in range(NUMBER_NODES):
            if i == j:
                continue
            elif i < j:
                dense_adj[i][j] = upperTriangleAdjMatrix[k]
                k += 1
            else:
                dense_adj[i][j] = dense_adj[j][i]
    return dense_adj  

# old version
# def saveAdjImg(graphInput, i, path):
#     graph_adj = getGraph(graphInput)
#     plt.imshow(graph_adj, cmap="gray")
#     plt.xticks([])
#     plt.yticks([])
#     plt.savefig(f'{path}/graph_{i}-{graphInput}.png')
#     plt.clf()

def saveImg(graphInput, idx, path):
    adj = getGraph(graphInput)
    w, h = NUMBER_NODES, NUMBER_NODES
    data = np.zeros((h, w, 3), dtype=np.uint8)
    for i in range(len(adj)):
        for j in range(len(adj)):
            if adj[i, j] == 1:
                data[i, j] = np.array([255.0, 255.0, 255.0])
    img = Image.fromarray(data, 'RGB')
    resized = img.resize((32, 32), Image.NEAREST)
    resized.save(f'{path}/graph_{idx}.png')

def getLabelStr(labels):
    labels = labels.replace('[', '')
    labels = labels.replace(']', '')
    labels = labels.replace('.', '')
    labels = labels.replace(' ', '')
    return labels

In [3]:
featuresNumber = ((NUMBER_NODES * NUMBER_NODES - NUMBER_NODES) // 2 )
train_df = pd.read_csv(os.path.join('..', 'DNN', 'datasets', f'dataset_{NUMBER_NODES}_train.csv'))
val_df = pd.read_csv(os.path.join('..', 'DNN', 'datasets', f'dataset_{NUMBER_NODES}_val.csv'))
test_df = pd.read_csv(os.path.join('..', 'DNN', 'datasets', f'dataset_{NUMBER_NODES}_test.csv'))

train_df = pd.concat((train_df, val_df))

In [4]:
i = 0
for entry in train_df.values:
    features = entry[ : featuresNumber]
    labels = entry[ featuresNumber + 1: ]
    labels = getLabelStr(str(labels))
    path = f'./train_adj_matrix_{NUMBER_NODES}/{labels}'
    if not os.path.exists(path):
        os.makedirs(path)
    img_arr = saveImg(features, i, path)
    i += 1

path = f'./test_adj_matrix_{NUMBER_NODES}'
if not os.path.exists(path):
    os.makedirs(path)
test_df.to_csv(f'{path}/test_{NUMBER_NODES}.csv', index=False)

path = f'{path}/test_images'
if not os.path.exists(path):
    os.makedirs(path)

i = 0
for entry in test_df.values:
    features = entry[ : featuresNumber]
    labels = entry[ featuresNumber + 1: ]
    labels = getLabelStr(str(labels))
    path = f'./test_adj_matrix_{NUMBER_NODES}/test_images/{labels}'
    if not os.path.exists(path):
        os.makedirs(path)
    img_arr = saveImg(features, i, path)
    i += 1

In [5]:
# approach 1

# G = nx.Graph()
# G.add_edges_from([(1, 2), (1, 3)])
# adj = nx.to_numpy_matrix(G)

# plt.imshow(adj, cmap="gray")
# plt.xticks([])
# plt.yticks([])
# plt.show()

# approach 2

# w, h = 3, 3
# data = np.zeros((h, w, 3), dtype=np.uint8)
# for i in range(len(adj)):
#     for j in range(len(adj)):
#         if adj[i, j] == 1:
#             # data[i:170, j:170] = np.array([255, 255, 255])
#             data[i, j] = np.array([255, 255, 255])
# # data[0:256, 0:256] = [255, 255, 255] # red patch in upper left
# img = Image.fromarray(data, 'RGB')
# fixed_height = 1024
# height_percent = (fixed_height / float(img.size[1]))
# width_size = int((float(img.size[0]) * float(height_percent)))
# resized = img.resize((512, 512), Image.NEAREST)
# print(resized.size)
# # resized.save('my.png')
# resized.show()

# https://www.holisticseo.digital/python-seo/resize-image/
# https://pillow.readthedocs.io/en/stable/reference/Image.html
# https://www.pluralsight.com/guides/importing-image-data-into-numpy-arrays